<h1> 1.Časť - prieskumná analýza </h1>

V tejto analýze budeme pracovať s dvoma datasetmi personal kde budu osobné informácie a other, ktorý obshuje namerané hodnty pacientov z cukrovkou a bez nej táto informácia sa nachádza v stplci class.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


In [ ]:
dataP = pd.read_csv("personal_train.csv", index_col=0)
dataP.info()

V datasete personal máme 5 atributov ako môzeme vidieť po príkaze info. Z toho jeden čiselný a 4 kategorické.

In [ ]:
dataO = pd.read_csv("other_train.csv", index_col=0) #, na_values =["??"]
dataO.info()

V datasete other máme 21 atributov. Z toho 14 čiselných a 7 kategorických.
V tomto datasete sa už nachádzajú aj nulové hodnoty, keďže počet hodnôt v jednotlivých atribútoch sa líši. Spustením príkazu v nasledujúcej bunke zistíme koľko udajov obsahuje nulové hodnoty.
Tiež sme si všimli, že sa tu nachádza o 50 údajov viac v rovnakých atributoch ako v prvom datasete.

In [ ]:
dataO.shape[0] - dataO.dropna().shape[0]

In [ ]:
dataP.head() 

Z ukážky dát sme zistili, že niektoré hodnoty veku sú sabotáž ale keďže máme aj dátum tak budeme vedieť podozrivé hodnoty bez podozrivého dátumu doplniť. 

In [ ]:
dataO.head()

Po vypísaní ukážky dát sme si všimli, že stĺpce education-num a class sú kategorické aj keď sú číselné. Preto zmeníme ich typ z float64 na object.
Všimli sme si tiež, že pri atribúte personal_info sa nachádzajú až tri údaje - rasa, národnosť, vzdelanie a rodinný stav. Tieto informácie budeme musieť rozdeliž pre ich podrobnejšiu analýzu.

In [ ]:
temp = dataO['personal_info'].str.split("\r\r\n|--|\\|", expand = True)
dataO['ethn'] = temp[0]
dataO['state'] = temp[1]
dataO['edu'] = temp[2]
dataO['status'] = temp[3]
dataO['fam_memb'] = temp[4] 
dataO.drop(['personal_info'], axis = 1)
dataO.head()

Ako sme skôr opisovali, rozhodli sme sa rozdeliť atribút personal_info na päť podatribútov.

In [ ]:
dataO = dataO.astype({'class':str,"education-num":str})

In [ ]:
dataP.describe()

In [ ]:
dataP.describe(exclude=np.number)

Describe nám ukázal len to čo už vieme, že z atribútu vek bude potrebne odstraniť prípadne neskôr doplniť podľa dátumu hodnoty, ktoré sú záporné a taktiež tie, ktoré sú väčšie ako sto keďže je nepravdepodobné, že bude niekto starší. Všetky ostatné hodnoty vyzerajú bzť v poriadku. Až na dátum, ktorý je niekedý zapísaný v inom formáte, čo bude tiež neskôr potrebné upraviť na jednotný formát.

In [ ]:
dataO.describe()

Decribe pre druhý dataset nám ukúzal, že niektoré namerané hodnoty môžu byť chybné nakoľko sú priveľké skoky medzi priemernými hodnotami a maximami/minimami, napr.: skewness_glucose (priemer 4.226, medián 0.376 a maximálna hodnota 68.1), capital-gain (priemer 1198.26, medián 0, max 99999), to isté skewness_oxygen, kurtosis_glucose, mean_oxygen a capital loss. Budeme musieť preto stanoviť nejaký rozsah, od ktorého budeme brať namerané hodnoty ako korektné a tie, ktoré budú mimo vylúčime ako outlierov.
Ďalej sme si všimli, že pri atribúte hours-per-week sú niektoré velmi zvláštne hodnoty, ako napríklad 1 alebo 99 hodín odpracovaných týždenne, preto budeme tiež musieť stanovič nejaký rozsah.

In [ ]:
dataO.describe(exclude=np.number)

In [ ]:
dataO.info()

In [ ]:
dataO.median()
#dat prec kategorické

In [ ]:
dataP.age.plot.hist(bins=30)

In [ ]:
dataRelAge = dataP[(dataP['age']>0) & (dataP['age'] < 100)]
dataIrelAge = dataP[(dataP['age']<0) | (dataP['age'] > 100)]

In [ ]:
dataRelAge.age.plot.hist(bins=30)

In [ ]:
dataIrelAge.age.plot.hist(bins = 30)

Pre porovnanie vekov sme sa pre vizuálne zobrazenie rozhodli využiť histogram pretože box plot bol veľmi neprehladný. Ako môžeme vidieť aj outlieri majú +- normálne rozdelenie čo môže znamenať, že sa jednalo o skutočné dáta. Aj keď to mohol byť tento prípad, nevieme akým spôsobom boli upravené, preto ich aj tak vylúčime.

In [ ]:
#co vsetko histogramovat z tychto dat? treba to nejako upravovat??
dataO.hist(column = ['std_oxygen'], bins=30)
dataO.hist(column = ['skewness_oxygen'], bins=30)
dataO.hist(column = ['mean_oxygen'], bins=30)
plt.show()

In [ ]:
boxplot = dataO.boxplot(column=['std_oxygen'])

In [ ]:
boxplot = dataO.boxplot(column=['skewness_oxygen'])

In [ ]:
boxplot = dataO.boxplot(column=['mean_oxygen'])

Tu sme si vykreslili grafy ku kyslíkovým meraniam. Zistili sme, že všetky majú gama rozdelenie. Pre porovnanie sme vykreslili aj box-ploty, v ktorých si vieme pozrieť outlierov. VZhladom na toto rozdelenie nevieme určite, ktorý z outlierov by sa mali vylúčiť. V neskoršej fáze bude možné upraviť gama na normálne rozdelenie a byť si istejší ohraničením.

In [ ]:
dataO.hist(column = ['std_glucose'], bins=30)
dataO.hist(column = ['skewness_glucose'], bins=30)
dataO.hist(column = ['mean_glucose'], bins=30)
dataO.hist(column = ['kurtosis_glucose'], bins=30)
plt.show()

In [ ]:
dataO.boxplot(column = ['std_glucose'])

In [ ]:
dataO.boxplot(column = ['skewness_glucose'])

In [ ]:
dataO.boxplot(column = ['mean_glucose'])

In [ ]:
dataO.boxplot(column = ['kurtosis_glucose'])

Rovnako ako pri kyslíku sme sa rozhodli v neskoršej fáze upraviť rozdelenia na normálne pre lepšie oddelenie dát, ktoré budeme chcieť použiť od dát, ktoré nie.
Chceli sme spraviť aj histogramové + plotboxové porovnanie pre capital-loss, capital-gain a fnlwgt pretože vyzerali, že by spolu mohli súvisieť. Neboli sme si však istý či spolu súvisia kedže sa v niektorých prípadoch objavovali hodnoty aj v loss a zároveň v gain, preto sme sa rozhodli zatial to nechať tak.

In [ ]:
#STATISTIKA - KATEGORICKE ATRIBUTY
#doplnanie dat? vysoke veky ale je datum narodenia doplnit z toho???????????
# pokial chceme spravit napr class - ma to problem s tým ze je to cislo, ako to obísť??
plt.figure(figsize = (20,8))
sns.countplot(y = 'occupation', data = dataO)
plt.show()

In [ ]:
sns.countplot(x = 'pregnant', data = dataO)

In [ ]:
plt.figure(figsize = (20,8))
sns.countplot(y = 'education-num', data = dataO)
plt.show()

In [ ]:
sns.countplot(x = 'class', data = dataO)

In [ ]:
sns.countplot(x = 'income', data = dataO)

In [ ]:
sns.countplot(y = 'workclass', data = dataO)

In [ ]:
sns.countplot(y = 'ethn', data = dataO)

In [ ]:
plt.figure(figsize = (20,8))
sns.countplot(y = 'state', data = dataO)
plt.show()

In [ ]:
sns.countplot(y = 'edu', data = dataO)

In [ ]:
sns.countplot(y = 'status', data = dataO)

In [ ]:
sns.countplot(y = 'fam_memb', data = dataO)

Z histogramov pre kategorické atribúty sme si všimli, že sa v dátach nachádzaju dva typy pre nevyplnenu hodnotu  - ? a nan, ktoré bude treba zmeniť na jednotnú hodnotu.
Ďalej bude treba upraviť dáta s rovnakým významom ale rozdielným zápisom v atribúte pregnant kde sa nachádza viacero tvarov false a true.

<h1>Párová analýza</h1>

In [ ]:
dataO = dataO.replace({'class': {'0.0': 'f', '1.0': 't'}})
sns.pairplot(dataO.dropna(), hue = 'class')
plt.show()

In [ ]:
plt.figure(figsize = (25,10))
sns.heatmap(dataO.corr(), annot = True, fmt = '.3f')
plt.show()

Cez heat map sme zistili atribúty, ktoré nesúvisia so žiadnymi s iných a rozhodli sme sa ich preto nevyužívať. Tieto atribúty sú : capital-gain, capital-loss, hours-per-week, fnlwgt.

Najvyššia pozitívna korelácia je medzi skewness_glucose a kurtosis_glucose 0.962 

In [ ]:
sns.lmplot(x="skewness_glucose", y="kurtosis_glucose",  hue = "class",markers=["o", "x"], data=dataO.dropna())
plt.show()

Ako možeme vidieť z grafu zavislosť medzi týmito atribútmi je skoro lineárna aj keď pri pacientoch bez cukrovky je trochu rozptýlená takže je možne, že sa tam nachádzajú nejakí outlieri.
Najvyššia pozitívna korelácia medzi atribútmi kyslika a glukózy je medzi std_oxygen a kurtosis_glucose 0.618

In [ ]:
sns.lmplot(x="std_oxygen", y="kurtosis_glucose",  hue = "class",markers=["o", "x"], data=dataO.dropna())
plt.show()

Na tomto grafe vidno, že pri pacientoch s cukrovkou závislosť je, ale pri pacientoch bez cukrvky sa hodnota kurtosis_glucose takmer nemení. Najvyššia negatívna korelácia je medzi mean_glucose a kurtosis_glucose -0.925

In [ ]:
sns.lmplot(x="mean_glucose", y="kurtosis_glucose",  hue = "class",markers=["o", "x"], data=dataO.dropna())
plt.show()

Z tohto grafu je tiež zjavne, že korelácia sa líši u pacientov s cukrovkou. Najvyššia negatívna korelácia medzi atribútmi kyslika a glukózy je medzi std_oxygen a mean_glucose 0.618

In [ ]:
sns.lmplot(x="mean_glucose", y="std_oxygen",  hue = "class",markers=["o", "x"], data=dataO.dropna())
plt.show()

Tento graf nám dokonca ukazuje, že závislosť týchto hodnôt u cukrovkárov a necukrovkárov nie je len odlišná ale dokonca opačná.

Podľa tabuľky pairplot na začiatku tejto časti anlýzy sme vybrali aj niektoré atribúty, pri ktorých najviac vidno, že ich hodnota je odlišna pre cukrovkárov a necukrovkárov.

In [ ]:
sns.boxplot('class', 'skewness_glucose', data=dataO.dropna())

In [ ]:
sns.boxplot('class', 'kurtosis_glucose', data=dataO.dropna())

In [ ]:
sns.boxplot('class', 'skewness_oxygen', data=dataO.dropna())

<h1>Formulácia a štatistické overenie hypotéz o dátach</h1>
Hypotéza 1: Rozdiel medzi hodnotami skewness_glucose u cukrovkárov a necukrovkárov je signifikantný.
Hypotéza 2: Rozdiel medzi hodnotami kurtosis_glucose u cukrovkárov a necukrovkárov je signifikantný.

To, že nejaký rozdiel tam je, vidíme aj na základe vizuálneho porovnania grafov v závere predchádzajúcej časti analýzy. Ak chceme overiť, či je signifikantný, potrebujeme použiť štatistický test.

 Vzťah medzi nimi je nezávislý (pacien vždy buď má alebo nemá cukrovku). Do úvahy teda pripadá t-test alebo Mann-Whitenyho U test. Overenie predpokladov t-testu:

In [ ]:
diabet = dataO.loc[dataO['class'] == 't', 'skewness_glucose']
diabet = diabet.dropna()
diabet.describe()

In [ ]:
sns.distplot(diabet)

In [ ]:
def identify_outliers(a):
    lower = a.quantile(0.25) - 1.5 * stats.iqr(a)
    upper = a.quantile(0.75) + 1.5 * stats.iqr(a)
    
    return a[(a > upper) | (a < lower)]

In [ ]:
diabetOut = identify_outliers(diabet)
diabetOut

In [ ]:
diabet = diabet.drop(diabetOut.index)
sns.distplot(diabet)

Toto rozdelenie nevyzerá ako normálne rozdelenie. Ale ešte si to musíme overiť Shapiro-Wilkovým testom normálnosti.

In [ ]:
stats.shapiro(diabet)

Keďže hodnota p je oveľa menšia ako 0.05 tak to určite nie je normálne rozdelenie.

In [ ]:
notDiabet = dataO.loc[dataO['class'] == 'f', 'skewness_glucose']
notDiabet = diabet.dropna()
sns.distplot(notDiabet)

In [ ]:
notDiabetOut = identify_outliers(diabet)
notDiabetOut

In [ ]:
stats.mannwhitneyu(diabet, notDiabet)

Keďže  p>0,001 , pravdepodobnosť chyby 1. rádu (že nulová hypotéza je pravdivá, a my ju nezamietame) je viac ako 1 promile. Nevieme povedať, že rozdiel medzi hodnotami skewness_glucose u cukrovkárov a necukrovkárov je signifikantný.